# ドリルの長さを調べる
### 工具の一番下に当たるピクセルを発見する

 - 工具の部分だけを切り出す
 - 2値化して、工具の部分だけ0になるようにする
 - 値が0の一番下のピクセルを探して、一番上からの距離を計算する

In [16]:
import cv2
import numpy as np


def find_bottom_black_pixel_coordinates(image_path):
    # 画像を読み込む
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

    # 画像を2値化する
    _, binary_image = cv2.threshold(image, 128, 255, cv2.THRESH_BINARY)

    # 画像の高さと幅を取得
    height, width = binary_image.shape

    # 画像を下から上にスキャンして、一番最初に見つかった黒いピクセルの座標を取得
    for y in range(height-1, -1, -1):
        for x in range(width):
            if binary_image[y, x] == 0:  # 黒いピクセルを見つけたら
                marked_image = cv2.cvtColor(
                    cv2.imread(image_path), cv2.COLOR_BGR2RGB)
                marked_image[y, x] = [255, 0, 0]  # 赤い印をつける
                cv2.imwrite('binary_image_with_marker.png',
                            cv2.cvtColor(binary_image, cv2.COLOR_GRAY2RGB))
                cv2.imwrite('original_image_with_marker.png',
                            cv2.cvtColor(marked_image, cv2.COLOR_RGB2BGR))

                return x, y  # 座標を返す

    # 2値化した画像に座標の印をつける

    # 画像表示

    # 黒いピクセルが見つからなかった場合
    return None


# 画像のパスを指定して座標を取得
image_path = './drill_forcus_sample.png'
coordinates = find_bottom_black_pixel_coordinates(image_path)

if coordinates:
    print(f"一番下の黒いピクセルの座標: x={coordinates[0]}, y={coordinates[1]}")

else:
    print("画像中に黒いピクセルが見つかりませんでした。")

一番下の黒いピクセルの座標: x=293, y=445


# ドリルの横幅を調べる
### 工具の横幅を調べる

 - 本当に工具の種類部分だけの画像にする(別のやつが入っていると横幅に引っかかる)
 - 2値化して、工具の部分だけ0になるようにする
 - 横軸で、値が0のピクセルを探して、一番左からの距離と一番右からの距離を計算する

In [8]:
import cv2
import numpy as np


def find_max_width(image_path):
    # 画像を読み込む
    original_image = cv2.imread(image_path)
    resized_image = cv2.resize(original_image, (500, 500))  # 必要に応じてリサイズ

    # 画像を2値化する
    gray_image = cv2.cvtColor(resized_image, cv2.COLOR_BGR2GRAY)
    _, binary_image = cv2.threshold(gray_image, 128, 255, cv2.THRESH_BINARY)

    # 最大横幅とそのY座標を初期化
    max_width = 0
    max_width_y = 0

    target_min_x = 0
    target_max_x = 0

    # 1行ずつ黒色の最小と最大のピクセルを探し、横幅を計算する
    for y in range(binary_image.shape[0]):
        black_pixels = np.where(binary_image[y, :] == 0)[0]
        if len(black_pixels) > 0:
            min_x = np.min(black_pixels)
            max_x = np.max(black_pixels)
            width = max_x - min_x
            if width > max_width:
                target_max_x = max_x
                target_min_x = min_x
                max_width = width
                max_width_y = y

    print("最大横幅:", max_width)

    # 最大横幅があるY座標にマークを付ける
    marked_image = resized_image.copy()
    cv2.line(marked_image, (target_min_x, max_width_y),
             (target_max_x, max_width_y), (0, 255, 0), 2)
    # 画像保存
    cv2.imwrite('marked_image_with_max_width.png', marked_image)
    print("画像を保存しました。")


# 画像のパスを指定して処理を実行
image_path = './drill_test2.png'  # 画像のパスを適切に指定してください
find_max_width(image_path)

最大横幅: 59
画像を保存しました。


# ドリルの種類を調べる
### 工具がドリルかタップか、種類を調べる

 - 工具の一部の部分だけをパターンとして切り出す
 - 工具の写真を撮ったときに、パターンと一致する部分があるかどうかを調べる
 - パターンマッチングで、ドリルかタップかの判別をするのか、穴のサイズを含めた工具の種類を判別するのかを調べる

In [14]:
import cv2
import numpy as np


def template_matching(template_path, target_path, threshold=0.8):
    # テンプレート画像の読み込み
    template = cv2.imread(template_path, 0)

    # 対象画像の読み込み
    target = cv2.imread(target_path, 0)

    # テンプレートマッチングの実行
    result = cv2.matchTemplate(target, template, cv2.TM_CCOEFF_NORMED)

    # 最大値とその位置を取得
    min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(result)

    # マッチング率が閾値以上の場合に判定
    if max_val >= threshold:
        # テンプレートが最もマッチした位置に矩形を描画
        h, w = template.shape
        top_left = max_loc
        bottom_right = (top_left[0] + w, top_left[1] + h)
        cv2.rectangle(target, top_left, bottom_right, 0, 2)

        # 結果の表示
        cv2.imwrite('Result_templatematching.png', target)
        print("テンプレートが見つかりました。")
    else:
        print("テンプレートは見つかりませんでした。")


if __name__ == "__main__":
    template_path = './tool_templete_tap.png'
    target_path = './imgc.png'

    template_matching(template_path, target_path)

テンプレートが見つかりました。
